In [1]:
import scanpy as sc
import pandas as pd
import seaborn as sns
import anndata as ad
import numpy as np

/Users/yijwang-admin/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# read in data
with open("../../../Data/ROSMAP_blood_gene_expression_monocyte_ACTL_04172023.csv") as your_data:
    adata_rosmap = ad.read_csv(your_data, delimiter=',')
adata_rosmap = adata_rosmap.T
    
with open("../../../Data/ANMerge_blood_rna_gene_expr_processed_XH_10112023.csv") as your_data:
    adata_adni = ad.read_csv(your_data, delimiter=',')
adata_adni = adata_adni.T
    
# read in label
Blood_Info = pd.read_csv("../../../Data/ROSMAP_blood_clinical_monocyte_ACTL_04172023.csv",header=0, index_col=0)

# read in label
Blood_Adni_Info = pd.read_csv("../../../Data/ANMerge_blood_rna_clinical_processed_XH_10112023.csv",header=0, index_col=0)

# read in projection from Monocle3
ADNI_Transferred_Label = pd.read_csv("../../../Data/Monocle_LabelTransfer_ROSMAP_ANMerge.csv")

In [3]:
SID = list(ADNI_Transferred_Label['X'])
PreLabel = list(ADNI_Transferred_Label['cell_type_fix'])
Mapping_ADNIID_PreLabel = dict(zip(SID, PreLabel))

In [4]:
#Mapping_ADNIID_PreLabel

In [5]:
ADNI_Transferred_Label

,Unnamed: 0,X,Label,Gender,Age,Size_Factor,data_set,cell_type_xfr,cell_type_fix
0,1,DCR00001_1,Asym AD,0,80,0.920870,query,NaN,Asym AD
1,2,DCR00004_1,Asym AD,1,78,0.923292,query,NaN,Asym AD
2,3,DCR00009_1,Asym AD,0,69,0.927422,query,Typical AD,Typical AD
3,4,DCR00013_1,Asym AD,1,82,0.927517,query,NaN,Typical AD
4,5,DCR00014_1,Typical AD,0,75,0.921701,query,NaN,Asym AD
...,...,...,...,...,...,...,...,...,...
686,687,TLSMCI607_1,Typical AD,0,72,1.081312,query,NaN,Asym AD
687,688,TLSMCI608_1,Asym AD,1,73,1.077134,query,NaN,Asym AD
688,689,TLSMCI609_1,Typical AD,0,63,0.929060,query,NaN,Typical AD
689,690,TLSMCI611_1,Asym AD,0,78,0.928300,query,NaN,Asym AD


In [6]:
import collections
# For rosmap blood
Blood_ROSMAP_Info = Blood_Info
BloodSampleID_Rosmap = list(adata_rosmap.obs.index)

SID = list(Blood_ROSMAP_Info.specimenID)
ADType = list(Blood_ROSMAP_Info.Group1)
Mapping = {'Control': 0, 'Typical AD': 1, 'Asym AD': 2, 'Low-NFT AD': 3}
frequency_rosmap = collections.Counter(ADType)
print(dict(frequency_rosmap))

Rosmap_Sample_CN = list()
Rosmap_Sample_TAD = list()
Rosmap_Sample_AAD = list()
for i in range(len(SID)):
    if ADType[i] == 'Control':
        Rosmap_Sample_CN.append(SID[i])
    if ADType[i] == 'Typical AD' or ADType[i] == 'Low-NFT AD':
        Rosmap_Sample_TAD.append(SID[i])
    if ADType[i] == 'Asym AD':
        Rosmap_Sample_AAD.append(SID[i])
#print(Rosmap_Sample_CN)
#print(Rosmap_Sample_TAD)
#print(Rosmap_Sample_AAD)

SType_Blood_Rosmap = list()
for i in range(len(BloodSampleID_Rosmap)):
    if BloodSampleID_Rosmap[i] in SID:
        indt = SID.index(BloodSampleID_Rosmap[i])
        tmp = Mapping[ADType[indt]]
        SType_Blood_Rosmap.append(tmp)
print(SType_Blood_Rosmap)
print(len(SType_Blood_Rosmap))

{'Typical AD': 46, 'Asym AD': 88, 'Control': 18, 'Low-NFT AD': 5}
[1, 2, 2, 0, 1, 2, 1, 0, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 0, 1, 2, 2, 2, 1, 1, 2, 1, 1, 1, 1, 2, 0, 1, 1, 2, 1, 2, 2, 2, 1, 2, 2, 1, 1, 2, 1, 0, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 0, 2, 2, 1, 2, 2, 2, 1, 0, 0, 2, 2, 2, 1, 1, 2, 2, 0, 2, 3, 2, 1, 0, 2, 2, 3, 1, 0, 0, 2, 2, 2, 2, 3, 0, 0, 2, 2, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 1, 1, 2, 2, 1, 2, 0, 1, 1, 2, 2, 1, 1, 2, 1, 2, 2, 0, 0, 1, 2, 2, 2, 2, 3, 1, 2, 2, 2, 3, 2, 2]
157


In [7]:
Adni_SID = list(Blood_Adni_Info.index.values)
Adni_Type = list(Blood_Adni_Info.Diagnosis)

SID_data = list(adata_adni.obs.index)

Adni_Type_Data = list()
for i in range(len(SID_data)):
    if SID_data[i] in Adni_SID:
        indt = Adni_SID.index(SID_data[i])
        tmp = Adni_Type[indt]
        Adni_Type_Data.append(tmp)
print(Adni_Type_Data)

['AD', 'MCI', 'CTL', 'CTL', 'CTL', 'CTL', 'CTL', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'MCI', 'CTL', 'AD', 'CTL', 'CTL', 'CTL', 'AD', 'CTL', 'CTL', 'CTL', 'AD', 'CTL', 'CTL', 'CTL', 'AD', 'CTL', 'AD', 'CTL', 'CTL', 'MCI', 'CTL', 'CTL', 'CTL', 'AD', 'CTL', 'CTL', 'CTL', 'AD', 'MCI', 'AD', 'CTL', 'CTL', 'CTL', 'AD', 'AD', 'MCI', 'AD', 'CTL', 'CTL', 'CTL', 'CTL', 'AD', 'CTL', 'AD', 'MCI', 'CTL', 'AD', 'CTL', 'MCI', 'CTL', 'MCI', 'CTL', 'CTL', 'CTL', 'CTL', 'CTL', 'MCI', 'AD', 'CTL', 'CTL', 'CTL', 'MCI', 'CTL', 'CTL', 'CTL', 'MCI', 'CTL', 'CTL', 'CTL', 'CTL', 'AD', 'CTL', 'CTL', 'AD', 'CTL', 'CTL', 'CTL', 'CTL', 'AD', 'CTL', 'AD', 'AD', 'MCI', 'AD', 'CTL', 'MCI', 'CTL', 'MCI', 'MCI', 'MCI', 'MCI', 'MCI', 'MCI', 'MCI', 'MCI', 'MCI', 'MCI', 'CTL', 'MCI', 'MCI', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'AD', 'A

In [8]:
import collections
# For rosmap blood
Blood_ROSMAP_Info = Blood_Info
BloodSampleID_Rosmap = list(adata_rosmap.obs.index)

SID = list(Blood_ROSMAP_Info.specimenID)
ADType = list(Blood_ROSMAP_Info.Group1)
Mapping = {'Control': 0, 'Typical AD': 1, 'Asym AD': 2, 'Low-NFT AD': 3}
frequency_rosmap = collections.Counter(ADType)
print(dict(frequency_rosmap))

Rosmap_Sample_CN = list()
Rosmap_Sample_TAD = list()
Rosmap_Sample_AAD = list()
for i in range(len(SID)):
    if ADType[i] == 'Control':
        Rosmap_Sample_CN.append(SID[i])
    if ADType[i] == 'Typical AD' or ADType[i] == 'Low-NFT AD':
        Rosmap_Sample_TAD.append(SID[i])
    if ADType[i] == 'Asym AD':
        Rosmap_Sample_AAD.append(SID[i])
#print(Rosmap_Sample_CN)
#print(Rosmap_Sample_TAD)
#print(Rosmap_Sample_AAD)

SType_Blood_Rosmap = list()
for i in range(len(BloodSampleID_Rosmap)):
    if BloodSampleID_Rosmap[i] in SID:
        indt = SID.index(BloodSampleID_Rosmap[i])
        tmp = Mapping[ADType[indt]]
        SType_Blood_Rosmap.append(tmp)
print(SType_Blood_Rosmap)
print(len(SType_Blood_Rosmap))

{'Typical AD': 46, 'Asym AD': 88, 'Control': 18, 'Low-NFT AD': 5}
[1, 2, 2, 0, 1, 2, 1, 0, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 0, 1, 2, 2, 2, 1, 1, 2, 1, 1, 1, 1, 2, 0, 1, 1, 2, 1, 2, 2, 2, 1, 2, 2, 1, 1, 2, 1, 0, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 0, 2, 2, 1, 2, 2, 2, 1, 0, 0, 2, 2, 2, 1, 1, 2, 2, 0, 2, 3, 2, 1, 0, 2, 2, 3, 1, 0, 0, 2, 2, 2, 2, 3, 0, 0, 2, 2, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 1, 1, 2, 2, 1, 2, 0, 1, 1, 2, 2, 1, 1, 2, 1, 2, 2, 0, 0, 1, 2, 2, 2, 2, 3, 1, 2, 2, 2, 3, 2, 2]
157


In [9]:
# convert predited label
Mapping = {'Control': 0, 'Typical AD': 1, 'Asym AD': 2, 'Low-NFT AD': 3}
y_pred = np.zeros(len(SID_data))
for i in range(len(SID_data)):
    y_pred[i] = Mapping[Mapping_ADNIID_PreLabel[SID_data[i]]]
    

In [10]:
Mapping = {'CTL': 0, 'AD': 1, 'MCI': 2}
y_true_lb = Adni_Type_Data
y_true = np.zeros(len(y_true_lb))
for i in range(len(y_true_lb)):
    y_true[i] = Mapping[y_true_lb[i]]

In [11]:
pred_ind_cn = np.where(y_pred==0)[0]
true_ind_cn = np.where(y_true==0)[0]
overlap = list(set(pred_ind_cn) & set(true_ind_cn))
if len(pred_ind_cn) == 0:
    print("precision of CN is 0")
else:
    print("precision of CN is ", len(overlap)/len(pred_ind_cn))

pred_ind_tad = np.where(y_pred==1)[0]
true_ind_tad = np.where(y_true==1)[0]
overlap = list(set(pred_ind_tad) & set(true_ind_tad))
print("precision of TAD is ", len(overlap)/len(pred_ind_tad))

pred_ind_aad = np.where(y_pred==2)[0]
true_ind_aad = np.where(y_true==2)[0]
overlap = list(set(pred_ind_aad) & set(true_ind_aad))
print("precision of AAD is ", len(overlap)/len(pred_ind_aad))

precision of CN is 0
precision of TAD is  0.32471264367816094
precision of AAD is  0.30029154518950435


In [12]:
pred_ind_cn = np.where(y_pred==0)[0]
true_ind_cn = np.where(y_true==0)[0]
overlap = list(set(pred_ind_cn) & set(true_ind_cn))
#if len(pred_ind_cn) == 0:
#    print("precision of CN is 0")
#else:
#    print("precision of CN is ", len(overlap)/len(pred_ind_cn))


pred_ind_aad = np.where(y_pred==2)[0]
true_ind_aad = np.where(y_true==2)[0]
overlap = list(set(pred_ind_aad) & set(true_ind_aad))
#print("precision of AAD is ", len(overlap)/len(pred_ind_aad))

pred_ind_tad = np.where(y_pred==1)[0]
true_ind_tad = np.where(y_true==1)[0]
true_ind_tad_aad = list(true_ind_aad) + list(true_ind_tad)
overlap = list(set(pred_ind_tad) & set(true_ind_tad_aad))
print("precision of TAD is ", len(overlap)/len(pred_ind_tad))

pred_ind_aad_cn = list(pred_ind_aad) #+ list(pred_ind_cn)
true_ind_aad_cn = list(true_ind_cn) + list(true_ind_aad)
overlap = list(set(pred_ind_aad_cn) & set(true_ind_aad_cn))
print("precision of AAD  is ", len(overlap)/len(pred_ind_aad_cn))

pred_ind_aad_cn = list(pred_ind_cn) #list(pred_ind_aad) + 
true_ind_aad_cn = list(true_ind_cn) + list(true_ind_aad)
overlap = list(set(pred_ind_aad_cn) & set(true_ind_aad_cn))
if len(pred_ind_aad_cn) == 0:
    print("precision of CN  is 0")
else:
    print("precision of CN  is ", len(overlap)/len(pred_ind_aad_cn))

pred_ind_lt = np.where(y_pred==3)[0]
true_ind_lt = np.where(y_true==1)[0]
true_ind_lt_aad = list(true_ind_aad) + list(true_ind_lt)
overlap = list(set(pred_ind_lt) & set(true_ind_lt_aad))
if len(pred_ind_lt) == 0:
    print("precision of LowTau is 0")
else:
    print("precision of LowTau is ", len(overlap)/len(pred_ind_lt))

precision of TAD is  0.6580459770114943
precision of AAD  is  0.6793002915451894
precision of CN  is 0
precision of LowTau is 0
